Here I was experimenting with other methods and other vectorizers - but afterall they did not manage to beat the original (TF-IDF + MNB). 
In general this notebook is worthless. I am leaving it as a fun-fact.


In [9]:

import pandas as pd 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from joblib import dump

df_train = pd.read_csv("/home/maciej/Desktop/project/team_project_1/project_data/cleaned_train_data_2.csv")

X_train, X_test, y_train, y_test = train_test_split(df_train['tweet'], df_train['label'], test_size=0.1,random_state=69)

vectorizer_TDF = TfidfVectorizer()
# vectorizer = CountVectorizer()
X_train_counts = vectorizer_TDF.fit_transform(X_train)
X_test_counts = vectorizer_TDF.transform(X_test)

clf = MultinomialNB()
clf.fit(X_train_counts, y_train)

#model_filename = 'project_models/model_MNB_2.joblib'
#dump(clf, model_filename)
#vectorizer_filename = 'project_models/tfidf_vectorizer_2.joblib'
#dump(vectorizer, vectorizer_filename)

y_pred = clf.predict(X_test_counts)
accuracy = accuracy_score(y_test, y_pred)
print("accuracy:", accuracy)
print("confusion matrix:\n", confusion_matrix(y_test, y_pred))
print("classification report:\n", classification_report(y_test, y_pred))

accuracy: 0.9069373942470389
confusion matrix:
 [[506  82]
 [ 28 566]]
classification report:
               precision    recall  f1-score   support

           0       0.95      0.86      0.90       588
           1       0.87      0.95      0.91       594

    accuracy                           0.91      1182
   macro avg       0.91      0.91      0.91      1182
weighted avg       0.91      0.91      0.91      1182



In [12]:
import numpy as np

In [10]:

clf_loaded = clf
vectorizer_loaded = vectorizer_TDF

testing_df = pd.read_csv("/home/maciej/Desktop/project/team_project_1/project_data/test_data_1.csv")

if 'tweet' in testing_df.columns and 'label' in testing_df.columns:
    X_test_transformed = vectorizer_loaded.transform(testing_df['tweet'])
    predictions = clf_loaded.predict(X_test_transformed)

    accuracy = accuracy_score(testing_df['label'], predictions)
    conf_matrix = confusion_matrix(testing_df['label'], predictions)
    class_report = classification_report(testing_df['label'], predictions)

    print("accuracy:", accuracy)
    print("aconfusion mtrix :\n", conf_matrix)
    print("C. Report :\n", class_report)
else:
    print("Error: Dataframe does not exist")

accuracy: 0.8
aconfusion mtrix :
 [[69 21]
 [15 75]]
C. Report :
               precision    recall  f1-score   support

           0       0.82      0.77      0.79        90
           1       0.78      0.83      0.81        90

    accuracy                           0.80       180
   macro avg       0.80      0.80      0.80       180
weighted avg       0.80      0.80      0.80       180



In [11]:
vectorizer_Count =  CountVectorizer()
X_train_counts = vectorizer_Count.fit_transform(X_train)
X_test_counts = vectorizer_Count.transform(X_test)

clf = MultinomialNB()
clf.fit(X_train_counts, y_train)

y_pred = clf.predict(X_test_counts)
accuracy = accuracy_score(y_test, y_pred)
print("accuracy:", accuracy)
print("confusion matrix:\n", confusion_matrix(y_test, y_pred))
print("classification report:\n", classification_report(y_test, y_pred))

vectorizer_loaded = vectorizer_Count

if 'tweet' in testing_df.columns and 'label' in testing_df.columns:
    X_test_transformed = vectorizer_loaded.transform(testing_df['tweet'])
    predictions = clf_loaded.predict(X_test_transformed)

    accuracy = accuracy_score(testing_df['label'], predictions)
    conf_matrix = confusion_matrix(testing_df['label'], predictions)
    class_report = classification_report(testing_df['label'], predictions)

    print("accuracy:", accuracy)
    print("aconfusion mtrix :\n", conf_matrix)
    print("C. Report :\n", class_report)
else:
    print("Error: Dataframe does not exist")


accuracy: 0.9111675126903553
confusion matrix:
 [[523  65]
 [ 40 554]]
classification report:
               precision    recall  f1-score   support

           0       0.93      0.89      0.91       588
           1       0.89      0.93      0.91       594

    accuracy                           0.91      1182
   macro avg       0.91      0.91      0.91      1182
weighted avg       0.91      0.91      0.91      1182

accuracy: 0.7722222222222223
aconfusion mtrix :
 [[66 24]
 [17 73]]
C. Report :
               precision    recall  f1-score   support

           0       0.80      0.73      0.76        90
           1       0.75      0.81      0.78        90

    accuracy                           0.77       180
   macro avg       0.77      0.77      0.77       180
weighted avg       0.77      0.77      0.77       180



In [22]:
import pandas as pd

df_train = pd.read_csv("/home/maciej/Desktop/project/team_project_1/project_data/cleaned_train_data_2.csv")

if 'tweet' in df_train.columns and 'label' in df_train.columns:
    df_train['label'] = '__label__' + df_train['label'].astype(str)
    df_train['fasttext_input'] = df_train['label'] + ' ' + df_train['tweet']
    df_train['fasttext_input'].to_csv('fasttext_train.txt', index=False, header=False)
else:
    print("Error: Dataframe does not have the necessary columns.")
import fasttext
model = fasttext.train_supervised(input='fasttext_train.txt')
model.save_model('model_filename.bin')
model = fasttext.train_supervised(input='fasttext_train.txt', lr=0.5, epoch=25, wordNgrams=2, minCount=1)


Read 0M words
Number of words:  22896
Number of labels: 2
Progress: 100.0% words/sec/thread: 1649296 lr:  0.000000 avg.loss:  0.152780 ETA:   0h 0m 0s
Read 0M words
Number of words:  22896
Number of labels: 2
Progress: 100.0% words/sec/thread: 1034243 lr:  0.000000 avg.loss:  0.015400 ETA:   0h 0m 0s


In [26]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
test_df = testing_df
actual_labels = test_df['label']
predictions = model.predict(test_df['tweet'].tolist())
predicted_labels = [int(label[0].replace('__label__', '')) for label in predictions[0]]  
accuracy = accuracy_score(actual_labels, predicted_labels)
conf_matrix = confusion_matrix(actual_labels, predicted_labels)
class_report = classification_report(actual_labels, predicted_labels)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)


Accuracy: 0.6166666666666667
Confusion Matrix:
 [[83  7]
 [62 28]]
Classification Report:
               precision    recall  f1-score   support

           0       0.57      0.92      0.71        90
           1       0.80      0.31      0.45        90

    accuracy                           0.62       180
   macro avg       0.69      0.62      0.58       180
weighted avg       0.69      0.62      0.58       180

